In [36]:
from __future__ import print_function
import sys
import IPython
print('IPython:', IPython.__version__)
import numpy
print('numpy:', numpy.__version__)
import pandas
print('pandas:', pandas.__version__)
import sklearn
print('scikit-learn:', sklearn.__version__)
print('Python version:', sys.version)

IPython: 7.34.0
numpy: 1.25.2
pandas: 1.5.3
scikit-learn: 1.2.2
Python version: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/DATA QUALITY MANAGEMENT')

In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/DATA QUALITY MANAGEMENT

/content/drive/My Drive/Colab Notebooks/DATA QUALITY MANAGEMENT


In [ ]:
import pandas as pd
import numpy as np
import random
import string
import re

In [ ]:
def generate_valid_name():
    valid_name = ''

    # Generate first name
    first_name = ''.join(random.choices(string.ascii_uppercase, k=1)) + ''.join(random.choices(string.ascii_lowercase, k=random.randint(2, 7)))
    valid_name += first_name

    # Generate middle name
    if random.random() < 0.5:
        middle_name = ''.join(random.choices(string.ascii_uppercase, k=1)) + ''.join(random.choices(string.ascii_lowercase, k=random.randint(2, 7)))
        valid_name += ' ' + middle_name

    # Generate last name
    last_name = ''.join(random.choices(string.ascii_uppercase, k=1)) + ''.join(random.choices(string.ascii_lowercase, k=random.randint(2, 7)))
    valid_name += ' ' + last_name

    return valid_name

def generate_invalid_name():
    invalid_chars = string.punctuation + string.digits
    invalid_name = ''

    # Generate first name
    first_name = ''.join(random.choices(string.ascii_lowercase, k=random.randint(3, 8)))
    if random.random() < 0.5:
        first_name = ''.join(random.choices(invalid_chars, k=random.randint(1, 3))) + first_name
    invalid_name += first_name

    # Generate middle name
    if random.random() < 0.5:
        middle_name = ''.join(random.choices(string.ascii_lowercase, k=random.randint(3, 8)))
        if random.random() < 0.5:
            middle_name = ''.join(random.choices(invalid_chars, k=random.randint(1, 3))) + middle_name
        invalid_name += ' ' + middle_name

    # Generate last name
    last_name = ''.join(random.choices(string.ascii_lowercase, k=random.randint(3, 8)))
    if random.random() < 0.5:
        last_name = ''.join(random.choices(invalid_chars, k=random.randint(1, 3))) + last_name
    invalid_name += ' ' + last_name

    # Add double space
    if random.random() < 0.3:
        invalid_name = invalid_name.replace(' ', '  ')

    return invalid_name

def validate_name(text):
    pattern = r'^[a-zA-Z]+(?: [a-zA-Z]+)*$'
    if re.match(pattern, text):
        return 1
    else:
        return 0

def make_teks_proper(text):
    # Replace special character and numbers in the middle of string with space
    cleaned_string = re.sub(r'[^A-Za-z\s]+', ' ', text)
    # Remove non alphabetic character except space
    cleaned_string = re.sub(r'\s{2,}', r' ', cleaned_string)
    return cleaned_string.strip()  # Remove space in the first and last string

def upperCaseCheck(name):
    pattern = r'^[A-Z][a-z\s]*$'
    return re.match(pattern, name) is not None


In [ ]:
data  = []
initial_label = []
check_type = []

for _ in range(700):
  data.append(generate_valid_name())
  initial_label.append(1)
  check_type.append('name')

for _ in range(700):
  data.append(generate_invalid_name())
  initial_label.append(1)
  check_type.append('name')

In [ ]:
dataset = pd.DataFrame({"text" : data, 'initial_label' : initial_label, 'data_type': check_type})
dataset.to_csv('dataset/name_dataset.csv')
dataset

,text,initial_label,data_type
0,Fqtfej Fpu,1,name
1,Eix Cocpyww Klaxytt,1,name
2,Qou Jyzooxs Shbqyyxy,1,name
3,Lueqbxz Pddsi,1,name
4,Htaew Yxtgjgk,1,name
...,...,...,...
1395,6*<pxwsvg nsjbtgti >joblj,1,name
1396,\:yqse aetvphup qojkfx,1,name
1397,khgtrvu yhb ;.uzztr,1,name
1398,maxl <[`nxelpkn .sxkdh,1,name


In [ ]:
dataset['label'] = dataset['text'].apply(lambda x: validate_name(x))
dataset['clean'] = dataset.apply(lambda x: make_teks_proper(str(x['text'])) if x['label'] == False else x['text'], axis=1)
dataset['check'] = dataset['clean'].apply(lambda x: validate_name(x))

dataset[['text','label','clean','check']]

,text,label,clean,check
0,Fqtfej Fpu,1,Fqtfej Fpu,1
1,Eix Cocpyww Klaxytt,1,Eix Cocpyww Klaxytt,1
2,Qou Jyzooxs Shbqyyxy,1,Qou Jyzooxs Shbqyyxy,1
3,Lueqbxz Pddsi,1,Lueqbxz Pddsi,1
4,Htaew Yxtgjgk,1,Htaew Yxtgjgk,1
...,...,...,...,...
1395,6*<pxwsvg nsjbtgti >joblj,0,pxwsvg nsjbtgti joblj,1
1396,\:yqse aetvphup qojkfx,0,yqse aetvphup qojkfx,1
1397,khgtrvu yhb ;.uzztr,0,khgtrvu yhb uzztr,1
1398,maxl <[`nxelpkn .sxkdh,0,maxl nxelpkn sxkdh,1


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

data = dataset
# Split Feature (X) and label (Y)
X = data['text'].values
Y = data['label'].values

# Splitting data Training and Testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from joblib import dump, load

# Data dan parameter
data = dataset
max_words = 1000

# Create Model
mlp_pipeline = Pipeline([
    ('vectorizer', CountVectorizer(max_features=max_words, analyzer='char')),
    ('clf', MLPClassifier(hidden_layer_sizes=(128, 64), activation='relu', solver='adam',
                          alpha=0.0001, random_state=42, max_iter=1000)),
])

# Train Model
history = mlp_pipeline.fit(X_train, Y_train)

# Model Evaluation
accuracy = mlp_pipeline.score(X_test, Y_test)
print("Accuracy:", accuracy)

# Classification Report
print("MLP Classification Report")
y_pred = mlp_pipeline.predict(X_test)
print(classification_report(Y_test, y_pred))

#Save models
dump(mlp_pipeline, 'model/name_mlp_model.joblib')

Accuracy: 0.9642857142857143
MLP Classification Report
              precision    recall  f1-score   support

           0       1.00      0.91      0.95       116
           1       0.94      1.00      0.97       164

    accuracy                           0.96       280
   macro avg       0.97      0.96      0.96       280
weighted avg       0.97      0.96      0.96       280



['model/name_mlp_model.joblib']

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Pipeline
lr_model_pipeline = Pipeline([
    ('vectorizer', CountVectorizer(max_features=max_words, analyzer='char')),
    ('lr_model', LogisticRegression())
])

# Train model
lr_model_pipeline.fit(X_train, Y_train)

# Predictions
Y_train_pred = lr_model_pipeline.predict(X_train)
Y_test_pred = lr_model_pipeline.predict(X_test)

# Calculate accuracy
train_accuracy = accuracy_score(Y_train, Y_train_pred)
test_accuracy = accuracy_score(Y_test, Y_test_pred)
print("Training Accuracy:", train_accuracy)
print("Validation Accuracy:", test_accuracy)

# Classification report
print("Classification Report Logistic Regression:")
print(classification_report(Y_test, Y_test_pred))

#Save Models
dump(lr_model_pipeline, 'model/name_lr_model.joblib')


Training Accuracy: 0.9526785714285714
Validation Accuracy: 0.9464285714285714
Classification Report Logistic Regression:
              precision    recall  f1-score   support

           0       1.00      0.87      0.93       116
           1       0.92      1.00      0.96       164

    accuracy                           0.95       280
   macro avg       0.96      0.94      0.94       280
weighted avg       0.95      0.95      0.95       280



['model/name_lr_model.joblib']

In [ ]:
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Pipeline
xgb_model_pipeline = Pipeline([
    ('vectorizer', CountVectorizer(max_features=max_words, analyzer='char')),
    ('XGB_model', xgb.XGBClassifier())
])

# Train model
xgb_model_pipeline.fit(X_train, Y_train)

# Predictions
Y_train_pred = xgb_model_pipeline.predict(X_train)
Y_test_pred = xgb_model_pipeline.predict(X_test)

# Calculate accuracy
train_accuracy = accuracy_score(Y_train, Y_train_pred)
test_accuracy = accuracy_score(Y_test, Y_test_pred)
print("Training Accuracy:", train_accuracy)
print("Validation Accuracy:", test_accuracy)

# Classification report
print("Classification Report XGBoost:")
print(classification_report(Y_test, Y_test_pred))

#Save Models
dump(xgb_model_pipeline, 'model/name_xgb_model.joblib')


Training Accuracy: 1.0
Validation Accuracy: 0.9178571428571428
Classification Report XGBoost:
              precision    recall  f1-score   support

           0       0.98      0.82      0.89       116
           1       0.89      0.99      0.93       164

    accuracy                           0.92       280
   macro avg       0.93      0.90      0.91       280
weighted avg       0.92      0.92      0.92       280



['model/name_xgb_model.joblib']

In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Pipeline
svm_model_pipeline = Pipeline([
    ('vectorizer', CountVectorizer(max_features=max_words, analyzer='char')),
    ('SVC_model', SVC(kernel='linear'))
])

# Train model
svm_model_pipeline.fit(X_train, Y_train)

# Predictions
Y_train_pred = svm_model_pipeline.predict(X_train)
Y_test_pred = svm_model_pipeline.predict(X_test)

# Calculate accuracy
train_accuracy = accuracy_score(Y_train, Y_train_pred)
test_accuracy = accuracy_score(Y_test, Y_test_pred)
print("Training Accuracy:", train_accuracy)
print("Validation Accuracy:", test_accuracy)

# Classification report
print("Classification Report SVM:")
print(classification_report(Y_test, Y_test_pred))

#Save Models
dump(svm_model_pipeline, 'model/name_svm_model.joblib')


Training Accuracy: 0.9741071428571428
Validation Accuracy: 0.9678571428571429
Classification Report SVM:
              precision    recall  f1-score   support

           0       1.00      0.92      0.96       116
           1       0.95      1.00      0.97       164

    accuracy                           0.97       280
   macro avg       0.97      0.96      0.97       280
weighted avg       0.97      0.97      0.97       280



['model/name_svm_model.joblib']

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Pipeline
dt_model_pipeline = Pipeline([
    ('vectorizer', CountVectorizer(max_features=max_words, analyzer='char')),
    ('DT_model', DecisionTreeClassifier())
])

# Train model
dt_model_pipeline.fit(X_train, Y_train)

# Predictions
Y_train_pred = dt_model_pipeline.predict(X_train)
Y_test_pred = dt_model_pipeline.predict(X_test)

# Calculate accuracy
train_accuracy = accuracy_score(Y_train, Y_train_pred)
test_accuracy = accuracy_score(Y_test, Y_test_pred)
print("Training Accuracy:", train_accuracy)
print("Validation Accuracy:", test_accuracy)

# Classification report
print("Classification Report DT:")
print(classification_report(Y_test, Y_test_pred))

#Save Models
dump(dt_model_pipeline, 'model/name_dt_model.joblib')


Training Accuracy: 1.0
Validation Accuracy: 0.9392857142857143
Classification Report DT:
              precision    recall  f1-score   support

           0       0.90      0.96      0.93       116
           1       0.97      0.93      0.95       164

    accuracy                           0.94       280
   macro avg       0.94      0.94      0.94       280
weighted avg       0.94      0.94      0.94       280



['model/name_dt_model.joblib']

In [ ]:
# To be Predicted
tobe_predict = ["Juan $Cesar",
                "Mark .A Junior",
                "Park Ji Sung",
                'Harrison & Hayes',
                "Leonard 1lyas"]

# Load Model
name_svm_model = load('model/name_svm_model.joblib')
name_dt_model = load('model/name_dt_model.joblib')
name_lr_model = load('model/name_lr_model.joblib')
name_mlp_model = load('model/name_mlp_model.joblib')

#Prediction
print("SVM : ", name_svm_model.predict(tobe_predict))
print("LR : ", name_lr_model.predict(tobe_predict))
print("DT : ", name_dt_model.predict(tobe_predict))
print("MLP : ", name_mlp_model.predict(tobe_predict))

SVM :  [0 1 1 0 0]
LR :  [0 1 1 0 1]
DT :  [0 0 1 0 0]
MLP :  [0 1 1 0 0]
